In [22]:
import os
import librosa
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import librosa
import librosa.display
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, concatenate
from keras.optimizers import Adam
from PIL import Image


import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [2]:
working_OS = 'Windows'

if working_OS == 'MacOS':
    os.chdir(r"/Users/jordanlee/Code/School/CSCI416/music-genre-classification")
    print("Current Working Directory:", os.getcwd())

elif working_OS == 'Windows':
    os.chdir(r"C:\Code\School\CSCI416\music_genre_classification\music-genre-classification")
    print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Code\School\CSCI416\music_genre_classification\music-genre-classification


# Data Input

In [3]:
df_raw = pd.read_csv('data/features/features_cleaned.csv')
df_aug = pd.read_csv('data/features/aug_features_cleaned.csv')

In [9]:
label_encoder =LabelEncoder()
scaler = StandardScaler()

### Raw DF

In [137]:
X_raw = df_raw.drop(columns=['label'])
y_raw = df_raw['label']

y_raw_encoded = label_encoder.fit_transform(y_raw)
X_raw_scaled = scaler.fit_transform(X_raw)
X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X_raw_scaled, y_raw_encoded, test_size=0.2, random_state=42)

### Aug DF

In [138]:
X_aug = df_aug.drop(columns=['label'])
y_aug = df_aug['label']

y_aug_encoded = label_encoder.fit_transform(y_aug)
X_aug_scaled = scaler.fit_transform(X_aug)
X_aug_train, X_aug_test, y_aug_train, y_aug_test = train_test_split(X_aug_scaled, y_aug_encoded, test_size=0.2, random_state=42)

### Dirs

In [139]:
spectrogram_raw_dir = 'data/spectrograms'
spectrogram_aug_dir = "data/aug_spectrograms"

### Staging Spectrograms

In [140]:
def load_spectrograms_and_labels(spectrogram_dir):
    spectrograms = []  # List to store spectrogram data
    labels = []        # List to store genre labels
    
    # Loop through all subdirectories (each representing a genre)
    for genre in os.listdir(spectrogram_dir):
        genre_folder = os.path.join(spectrogram_dir, genre)
        
        # Skip files, process only directories
        if os.path.isdir(genre_folder):
            for img_file in os.listdir(genre_folder):
                img_path = os.path.join(genre_folder, img_file)
                
                # Read the image using PIL (you can resize or preprocess here if needed)
                img = Image.open(img_path).convert('L')  # Convert to grayscale ('L')
                img = img.resize((128, 128))  # Resize to 128x128 (adjust as needed)
                
                # Convert image to numpy array
                img_array = np.array(img)
                
                # Append the image and its genre label
                spectrograms.append(img_array)
                labels.append(genre)  # The genre is the label for classification
    
    # Convert lists to numpy arrays
    X_spectrograms = np.array(spectrograms)
    y_labels = np.array(labels)
    
    return X_spectrograms, y_labels

In [141]:
# Load spectrograms and labels
X_raw_spectrograms, y_raw_spectrogram_labels = load_spectrograms_and_labels(spectrogram_raw_dir)
X_aug_spectrograms, y_aug_spectrogram_labels = load_spectrograms_and_labels(spectrogram_aug_dir)

# Normalize pixel values (between 0 and 1)
X_raw_spectrograms = X_raw_spectrograms.astype('float32') / 255.0
X_aug_spectrograms = X_aug_spectrograms.astype('float32') / 255.0

# Reshape to add channel dimension (since images are grayscale, the channel is 1)
X_raw_spectrograms = X_raw_spectrograms.reshape(-1, 128, 128, 1)
X_aug_spectrograms = X_aug_spectrograms.reshape(-1, 128, 128, 1)

# Encode labels (genres) as numeric values
y_raw_spectrogram_labels_encoded = label_encoder.fit_transform(y_raw_spectrogram_labels)
y_aug_spectrogram_labels_encoded = label_encoder.fit_transform(y_aug_spectrogram_labels)

# Optional: one-hot encode the labels if needed (for multi-class classification)
y_raw_labels_one_hot = np.eye(len(label_encoder.classes_))[y_raw_spectrogram_labels_encoded]
y_aug_labels_one_hot = np.eye(len(label_encoder.classes_))[y_aug_spectrogram_labels_encoded]

X_raw_train_CNN, X_raw_test_CNN, y_raw_train_CNN, y_raw_test_CNN = train_test_split(
    X_raw_spectrograms, y_raw_labels_one_hot, test_size=0.2, random_state=42
)

X_aug_train_CNN, X_aug_test_CNN, y_aug_train_CNN, y_aug_test_CNN = train_test_split(
    X_aug_spectrograms, y_aug_labels_one_hot, test_size=0.2, random_state=42
)

 ### Defining CNNs

In [142]:
# Initialize the ImageDataGenerator for augmentation
datagen_raw = ImageDataGenerator(
    rotation_range=30,       # Random rotations
    width_shift_range=0.2,   # Random width shifts
    height_shift_range=0.2,  # Random height shifts
    shear_range=0.2,         # Random shearing
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Random horizontal flip
    fill_mode='nearest'      # Fill missing pixels after transformations
)

datagen_aug = ImageDataGenerator(
    rotation_range=30,       # Random rotations
    width_shift_range=0.2,   # Random width shifts
    height_shift_range=0.2,  # Random height shifts
    shear_range=0.2,         # Random shearing
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Random horizontal flip
    fill_mode='nearest'      # Fill missing pixels after transformations
)

# Fit the generator on your training data
datagen_raw.fit(X_raw_train_CNN)

datagen_aug.fit(X_aug_train_CNN)

## Model Training

### Random Forest Models

#### Optimization

In [32]:
def optimize_rf_hyperparameters(X, y):
    # Create a pipeline that includes scaling and the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestClassifier(random_state=42))
    ])
    
    # Parameter grid for Random Forest
    param_dist = {
        'rf__n_estimators': [50, 100, 200],
        'rf__max_depth': [None, 10, 20, 30],
        'rf__min_samples_split': [2, 5, 10],
        'rf__min_samples_leaf': [1, 2, 4]
    }
    
    # Randomized search with cross-validation
    random_search = RandomizedSearchCV(
        pipeline, 
        param_distributions=param_dist,
        n_iter=20,
        cv=5,
        scoring='accuracy',
        random_state=42
    )
    
    # Fit on the original X and y (scaling happens inside the pipeline)
    random_search.fit(X, y)
    
    print("Best parameters:", random_search.best_params_)
    print("Best cross-validated score:", random_search.best_score_)
    
    return random_search.best_estimator_


In [28]:
optimize_rf_hyperparameters(X_raw_scaled, y_raw_encoded)

Best parameters: {'rf__n_estimators': 100, 'rf__min_samples_split': 2, 'rf__min_samples_leaf': 2, 'rf__max_depth': None}
Best cross-validated score: 0.6235728643216081


Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(min_samples_leaf=2, random_state=42))])

In [29]:
optimize_rf_hyperparameters(X_aug_scaled, y_aug_encoded)

Best parameters: {'rf__n_estimators': 200, 'rf__min_samples_split': 2, 'rf__min_samples_leaf': 1, 'rf__max_depth': 30}
Best cross-validated score: 0.5919304396215915


Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=30, n_estimators=200,
                                        random_state=42))])

In [48]:
rf_raw_model = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf= 2, max_depth= None, random_state=42)
rf_raw_model.fit(X_raw_train, y_raw_train)

RandomForestClassifier(min_samples_leaf=2, random_state=42)

In [55]:
rf_aug_model = RandomForestClassifier(n_estimators=200, min_samples_split=2, min_samples_leaf= 1, max_depth= 30, random_state=42)
rf_aug_model.fit(X_aug_train, y_aug_train)

RandomForestClassifier(max_depth=30, n_estimators=200, random_state=42)

### Support Vector Machines

#### Optimizing

In [50]:
def optimize_svm_hyperparameters(X, y):
    # Create a pipeline that includes scaling and SVM
    svm_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(probability=True, random_state=42))
    ])
    
    # Comprehensive parameter grid for SVM
    param_dist = {
        'svm__C': [0.1, 1, 10, 100],  # Regularization parameter
        'svm__kernel': ['rbf', 'linear', 'poly'],  # Kernel types
        'svm__gamma': ['scale', 'auto', 0.1, 0.01, 0.001],  # Kernel coefficient
        'svm__class_weight': [None, 'balanced'],
        'svm__degree': [2, 3, 4]  # For polynomial kernel
    }
    
    # Randomized search with cross-validation
    random_search = RandomizedSearchCV(
        svm_pipeline, 
        param_distributions=param_dist,
        n_iter=20,  # Number of parameter settings to sample
        cv=5,  # 5-fold cross-validation
        scoring='accuracy',
        random_state=42,
        verbose=2  # Show progress
    )
    
    # Fit on the data
    random_search.fit(X, y)
    
    # Print results
    print("Best SVM parameters:", random_search.best_params_)
    print("Best cross-validated score:", random_search.best_score_)
    
    return random_search.best_estimator_

In [35]:
best_raw_svm = optimize_svm_hyperparameters(X_raw_scaled, y_raw_encoded)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.0s
[CV] END svm__C=0.1, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=rbf; total time=   0.1s
[CV] END svm__C=0.1, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=rbf; total time=   0.1s
[CV] END svm__C=0.1, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=rbf; total time=   0.1s
[CV] END svm__C=0.1, svm__

In [36]:
best_aug_svm = optimize_svm_hyperparameters(X_aug_scaled, y_aug_encoded)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.8s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.8s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.8s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.8s
[CV] END svm__C=10, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=poly; total time=   0.8s
[CV] END svm__C=0.1, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=rbf; total time=   1.5s
[CV] END svm__C=0.1, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=rbf; total time=   1.5s
[CV] END svm__C=0.1, svm__class_weight=None, svm__degree=4, svm__gamma=0.001, svm__kernel=rbf; total time=   1.5s
[CV] END svm__C=0.1, svm__

In [51]:
svm_raw_model = SVC(kernel='rbf', gamma= 0.01, degree=4, class_weight='balanced', C=100, probability=True, random_state=42)
svm_raw_model.fit(X_raw_train, y_raw_train)

SVC(C=100, class_weight='balanced', degree=4, gamma=0.01, probability=True,
    random_state=42)

In [52]:
svm_aug_model = SVC(kernel='rbf', gamma= 0.01, degree=4, class_weight='balanced', C=100, probability=True, random_state=42)
svm_aug_model.fit(X_aug_train, y_aug_train)

SVC(C=100, class_weight='balanced', degree=4, gamma=0.01, probability=True,
    random_state=42)

In [40]:
raw_label_names = sorted(df_raw['label'].unique())
aug_label_names = sorted(df_aug['label'].unique())

### Convolutional Neural Network

In [148]:
# Had to import from another notebook because model was not correctly defined here
small_raw_CNN_model = joblib.load('models/small_raw_CNN_model.joblib')
large_raw_CNN_model = joblib.load('models/large_raw_CNN_model.joblib')
small_aug_CNN_model = joblib.load('models/small_aug_CNN_model.joblib')
large_aug_CNN_model = joblib.load('models/large_raw_CNN_model.joblib')

In [154]:
small_raw_CNN_model.fit(
    [X_raw_train, X_raw_train_CNN], y_raw_train_CNN, 
    epochs=10, 
    batch_size=32, 
    validation_data=([X_raw_test, X_raw_test_CNN], y_raw_test_CNN)
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 1.0000 - loss: 2.9606e-04 - val_accuracy: 0.6750 - val_loss: 1.6805
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 1.0000 - loss: 2.7858e-04 - val_accuracy: 0.6850 - val_loss: 1.6847
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 1.0000 - loss: 2.5348e-04 - val_accuracy: 0.6700 - val_loss: 1.6955
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 1.0000 - loss: 2.5032e-04 - val_accuracy: 0.6750 - val_loss: 1.7055
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 1.0000 - loss: 2.3814e-04 - val_accuracy: 0.6700 - val_loss: 1.7090
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 1.0000 - loss: 2.2132e-04 - val_accuracy: 0.6800 - val_loss: 1.7210
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 1.0000 - loss: 2.2108e-04 - val_accuracy: 0.6750 - val_loss: 1.7233
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 1.0000 - loss: 2.073

In [155]:
large_raw_CNN_model.fit(
    [X_raw_train, X_raw_train_CNN], y_raw_train_CNN, 
    epochs=10, 
    batch_size=32, 
    validation_data=([X_raw_test, X_raw_test_CNN], y_raw_test_CNN)
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 358ms/step - accuracy: 1.0000 - loss: 0.0110 - val_accuracy: 0.2100 - val_loss: 3.1642
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 352ms/step - accuracy: 1.0000 - loss: 0.0092 - val_accuracy: 0.2000 - val_loss: 3.2880
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 356ms/step - accuracy: 1.0000 - loss: 0.0086 - val_accuracy: 0.2650 - val_loss: 2.9804
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 354ms/step - accuracy: 1.0000 - loss: 0.0081 - val_accuracy: 0.3400 - val_loss: 2.4120
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 358ms/step - accuracy: 1.0000 - loss: 0.0089 - val_accuracy: 0.4150 - val_loss: 1.9653
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 357ms/step - accuracy: 1.0000 - loss: 0.0082 - val_accuracy: 0.3150 - val_loss: 2.2478
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 356ms/step - accuracy: 0.9974 - loss: 0.0139 - val_accuracy: 0.3200 - val_loss: 2.8242
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 357ms/step - accuracy: 1.0000 - loss: 0.0096 - val_accuracy: 0

In [152]:
small_aug_CNN_model.fit(
    [X_aug_train, X_aug_train_CNN], y_aug_train_CNN, 
    epochs=10, 
    batch_size=32, 
    validation_data=([X_raw_test, X_raw_test_CNN], y_raw_test_CNN)
)

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 22s 206ms/step - accuracy: 0.9629 - loss: 0.1583 - val_accuracy: 0.1100 - val_loss: 20.6191
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 192ms/step - accuracy: 0.9700 - loss: 0.1230 - val_accuracy: 0.1100 - val_loss: 23.1254
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 90ms/step - accuracy: 0.9826 - loss: 0.0843 - val_accuracy: 0.1250 - val_loss: 16.8042
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 88ms/step - accuracy: 0.9978 - loss: 0.0393 - val_accuracy: 0.1300 - val_loss: 17.2483
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 87ms/step - accuracy: 0.9988 - loss: 0.0224 - val_accuracy: 0.1200 - val_loss: 22.2582
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 1.0000 - loss: 0.0111 - val_accuracy: 0.1250 - val_loss: 20.2400
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - accuracy: 1.0000 - loss: 0.0084 - val_accuracy: 0.1200 - val_loss: 22.1465
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 87ms/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy:

In [153]:
large_aug_CNN_model.fit(
    [X_aug_train, X_aug_train_CNN], y_aug_train_CNN, 
    epochs=10, 
    batch_size=32, 
    validation_data=([X_raw_test, X_raw_test_CNN], y_raw_test_CNN)
)

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 28s 345ms/step - accuracy: 0.2992 - loss: 2.4158 - val_accuracy: 0.0900 - val_loss: 21.3790
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 340ms/step - accuracy: 0.5416 - loss: 1.3474 - val_accuracy: 0.1400 - val_loss: 25.8267
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 345ms/step - accuracy: 0.5782 - loss: 1.2048 - val_accuracy: 0.1350 - val_loss: 6.6589
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 27s 362ms/step - accuracy: 0.6981 - loss: 0.8946 - val_accuracy: 0.1350 - val_loss: 7.0838
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 29s 381ms/step - accuracy: 0.7779 - loss: 0.6563 - val_accuracy: 0.1350 - val_loss: 11.1721
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 349ms/step - accuracy: 0.8351 - loss: 0.5166 - val_accuracy: 0.1350 - val_loss: 26.5227
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 346ms/step - accuracy: 0.8419 - loss: 0.4696 - val_accuracy: 0.1150 - val_loss: 17.0164
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 347ms/step - accuracy: 0.8910 - loss: 0.3588 - val

### Accuracy Testing

##### RF

In [110]:
y_pred_rf_raw = rf_raw_model.predict(X_raw_test)
print(classification_report(y_raw_test, y_pred_rf_raw))
print("Accuracy:", accuracy_score(y_raw_test, y_pred_rf_raw))

              precision    recall  f1-score   support

           0       0.89      0.81      0.85        21
           1       0.75      1.00      0.86        12
           2       0.78      0.75      0.77        24
           3       0.95      0.82      0.88        22
           4       0.71      0.80      0.75        15
           5       0.96      0.93      0.94        27
           6       0.86      1.00      0.92        18
           7       0.81      0.89      0.85        19
           8       0.76      0.73      0.74        22
           9       0.59      0.50      0.54        20

    accuracy                           0.81       200
   macro avg       0.81      0.82      0.81       200
weighted avg       0.82      0.81      0.81       200

Accuracy: 0.815


In [170]:
y_pred_rf_aug = rf_aug_model.predict(X_raw_test)
print(classification_report(y_raw_test, y_pred_rf_aug))
print("Accuracy:", accuracy_score(y_raw_test, y_pred_rf_aug))

              precision    recall  f1-score   support

           0       0.95      0.86      0.90        21
           1       0.67      1.00      0.80        12
           2       0.91      0.83      0.87        24
           3       0.94      0.73      0.82        22
           4       0.81      0.87      0.84        15
           5       0.96      0.85      0.90        27
           6       0.78      1.00      0.88        18
           7       0.86      1.00      0.93        19
           8       0.91      0.91      0.91        22
           9       0.76      0.65      0.70        20

    accuracy                           0.86       200
   macro avg       0.86      0.87      0.85       200
weighted avg       0.87      0.86      0.86       200

Accuracy: 0.86


##### SVM

In [112]:
y_pred_svm_raw = svm_raw_model.predict(X_raw_test)
print(classification_report(y_raw_test, y_pred_svm_raw))
print("Accuracy:", accuracy_score(y_raw_test, y_pred_svm_raw))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79        21
           1       0.85      0.92      0.88        12
           2       0.70      0.79      0.75        24
           3       0.65      0.68      0.67        22
           4       0.71      0.80      0.75        15
           5       0.88      0.85      0.87        27
           6       0.88      0.83      0.86        18
           7       0.89      0.84      0.86        19
           8       0.68      0.59      0.63        22
           9       0.50      0.45      0.47        20

    accuracy                           0.75       200
   macro avg       0.75      0.76      0.75       200
weighted avg       0.75      0.75      0.75       200

Accuracy: 0.75


In [171]:
y_pred_svm_aug = svm_aug_model.predict(X_raw_test)
print(classification_report(y_raw_test, y_pred_svm_aug))
print("Accuracy:", accuracy_score(y_raw_test, y_pred_svm_aug))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        21
           1       0.92      0.92      0.92        12
           2       0.96      1.00      0.98        24
           3       1.00      0.95      0.98        22
           4       1.00      0.93      0.97        15
           5       0.96      1.00      0.98        27
           6       0.86      1.00      0.92        18
           7       0.90      0.95      0.92        19
           8       0.95      0.95      0.95        22
           9       1.00      0.80      0.89        20

    accuracy                           0.95       200
   macro avg       0.95      0.95      0.95       200
weighted avg       0.95      0.95      0.95       200

Accuracy: 0.95


##### CNN

In [162]:
test_small_raw_loss, test_small_raw_accuracy = small_raw_CNN_model.evaluate([X_raw_test, X_raw_test_CNN], y_raw_test_CNN)

# Print the results
print(f"Test loss: {test_small_raw_loss}")
print(f"Test accuracy: {test_small_raw_accuracy}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6598 - loss: 2.0144
Test loss: 1.738380789756775
Test accuracy: 0.675000011920929


In [163]:
test_large_raw_loss, test_large_raw_accuracy = large_raw_CNN_model.evaluate([X_raw_test, X_raw_test_CNN], y_raw_test_CNN)

# Print the results
print(f"Test loss: {test_large_raw_loss}")
print(f"Test accuracy: {test_large_raw_accuracy}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5046 - loss: 1.7746
Test loss: 1.7215609550476074
Test accuracy: 0.5149999856948853


In [164]:
test_small_aug_loss, test_small_aug_accuracy = small_aug_CNN_model.evaluate([X_aug_test, X_aug_test_CNN], y_aug_test_CNN)

# Print the results
print(f"Test loss: {test_small_aug_loss}")
print(f"Test accuracy: {test_small_aug_accuracy}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8154 - loss: 0.9251
Test loss: 0.8583638668060303
Test accuracy: 0.8016666769981384


In [165]:
test_large_aug_loss, test_large_aug_accuracy = large_aug_CNN_model.evaluate([X_aug_test, X_aug_test_CNN], y_aug_test_CNN)

# Print the results
print(f"Test loss: {test_large_aug_loss}")
print(f"Test accuracy: {test_large_aug_accuracy}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.7681 - loss: 0.7024
Test loss: 0.5930794477462769
Test accuracy: 0.8050000071525574
